# Capstone Project

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem<a name="introduction"></a>

Intrepid Travel is a company based in Las Vegas for tourism activities and spots, currently it is one of the highest rated Dallas-based companies for tourism. However, it can be difficult to manually plan and find out all of the available places in Las Vegas for each activity and can be an uneccessary use of capital. Therefore, it would be helpful to have a model to plan out where important locations are. The company has to put a good tourism program and package, including a hotel, places of landscape to visit, stores for shopping, restaurants and cafes, and tourism attractions. So the company’s purpose is to make a list of places of landscape in Dallas, including the nearest restaurants, cafes, and shopping stores, and tourism attractions for each place.

## Data <a name="data"></a>

The data used in this project is provided by Foursquare location data. The data are grouped by landscape area, and each area included the information about this area and all information about restaurants, cafes, tourism attractions, and stores which are located in this area. The only data that is simply needed is the data for different types of places, attractions, and edifices of purpose. After this, our goal is simply to map the results. Let's import all the libraries first.

### First, let's import all the libraries that we will need.

In [61]:
import requests # to handle requests
import pandas as pd # for data analsysis
import numpy as np # to handle data in a vectorized manner
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import folium # plotting library

### Next, let's establish a secure connection to the FourSquare API as we will be using it for the majority of this study

In [62]:
ClIENT_ID = '2XM3I5ZAP13OTFTXRDRE03CUCDA5LWAPRFVJXX5BPO0CIOWX' # your Foursquare ID
ClIENT_SECRET = 'GK0UWICKTRQ4QPXTWV5DFBLVZQ3WJGZHDLX31OMRRNK0ZYT2' # your Foursquare Secret
VERSION = '20200425'
LIMIT =30
print('Your credentails:')
print('Foursquare_ID: ' + ClIENT_ID)
print('Foursquare_Secret:' + ClIENT_SECRET)

Your credentails:
Foursquare_ID: 2XM3I5ZAP13OTFTXRDRE03CUCDA5LWAPRFVJXX5BPO0CIOWX
Foursquare_Secret:GK0UWICKTRQ4QPXTWV5DFBLVZQ3WJGZHDLX31OMRRNK0ZYT2


### Since the city Intrepid Travels operates in is Las Vegas, let's pull the absolute position of it in order to use later in the project.

In [63]:
# define the city and get its latitude & longitude 
city = 'Las Vegas'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

36.1672559 -115.1485163


### Because the most important touristic activity for long term stayers is the finding and booking of a hotel, let's first focus on finding hotels in the Las Vegas metroplex.

In [64]:
# search for hotels
search_query = 'Hotel'
radius = 500

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=2XM3I5ZAP13OTFTXRDRE03CUCDA5LWAPRFVJXX5BPO0CIOWX&client_secret=GK0UWICKTRQ4QPXTWV5DFBLVZQ3WJGZHDLX31OMRRNK0ZYT2&ll=36.1672559,-115.1485163&v=20200425&query=Hotel&radius=500&limit=30'

In [65]:
# Send the GET Request and examine the results
results = requests.get(url).json()

In [66]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

C:\Users\vkogi\Anaconda3\envs\python-cvcourse\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,45a90a24f964a52020411fe3,Golden Nugget Hotel & Casino,"[{'id': '4bf58dd8d48988d17c941735', 'name': 'C...",v-1587857251,False,129 Fremont St,at S Casino Center Blvd,36.170222,-115.144969,"[{'label': 'display', 'lat': 36.170222, 'lng':...",458,89101,US,Las Vegas,NV,United States,"[129 Fremont St (at S Casino Center Blvd), Las...",NaN
1,4ba3a36bf964a520b24e38e3,William Hill Sports Book - The Plaza Hotel and...,"[{'id': '4bf58dd8d48988d17c941735', 'name': 'C...",v-1587857251,False,1 S Main St,Fremont Street,36.171239,-115.146849,"[{'label': 'display', 'lat': 36.17123901693208...",468,89101,US,Las Vegas,NV,United States,"[1 S Main St (Fremont Street), Las Vegas, NV 8...",NaN
2,50a994b0e4b0534d8e4eb26f,Golden Gate Hotel & Casino,"[{'id': '4bf58dd8d48988d17c941735', 'name': 'C...",v-1587857251,False,1 Fremont Street,Fremont Street Experience,36.171263,-115.146209,"[{'label': 'display', 'lat': 36.17126267, 'lng...",491,89101,US,Las Vegas,NV,United States,"[1 Fremont Street (Fremont Street Experience),...",556015737
3,4edd4eb1754a23a1927a192e,LasVegasShows.com Ticket Kiosk - Plaza Hotel a...,"[{'id': '4bf58dd8d48988d1ff941735', 'name': 'M...",v-1587857251,False,1 S Main St,NaN,36.171863,-115.146721,"[{'label': 'display', 'lat': 36.17186295186981...",537,89101,US,Las Vegas,NV,United States,"[1 S Main St, Las Vegas, NV 89101, United States]",NaN
4,565f0c61498e5954cc590d8a,Hollywood Palace Hotel (PieceOfMe),"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1587857251,False,NaN,NaN,36.168302,-115.145427,"[{'label': 'display', 'lat': 36.168302, 'lng':...",301,NaN,US,Las Vegas,NV,United States,"[Las Vegas, NV, United States]",NaN


### Now that we have found the hotels within a 500 radius of Las Vegas, Nevada, lets clean up the table and make sure we are only working with relevant information like hotel info and location info. Let's also make sure the category can be clearly seen.

In [71]:
# keep only columns that include venue name, and anything that is associated with location
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]
# delete unnecessary columns
clean_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Golden Nugget Hotel & Casino,Casino,129 Fremont St,at S Casino Center Blvd,36.170222,-115.144969,"[{'label': 'display', 'lat': 36.170222, 'lng':...",458,89101,US,Las Vegas,NV,United States,"[129 Fremont St (at S Casino Center Blvd), Las...",45a90a24f964a52020411fe3
1,William Hill Sports Book - The Plaza Hotel and...,Casino,1 S Main St,Fremont Street,36.171239,-115.146849,"[{'label': 'display', 'lat': 36.17123901693208...",468,89101,US,Las Vegas,NV,United States,"[1 S Main St (Fremont Street), Las Vegas, NV 8...",4ba3a36bf964a520b24e38e3
2,Golden Gate Hotel & Casino,Casino,1 Fremont Street,Fremont Street Experience,36.171263,-115.146209,"[{'label': 'display', 'lat': 36.17126267, 'lng...",491,89101,US,Las Vegas,NV,United States,"[1 Fremont Street (Fremont Street Experience),...",50a994b0e4b0534d8e4eb26f
3,LasVegasShows.com Ticket Kiosk - Plaza Hotel a...,Miscellaneous Shop,1 S Main St,NaN,36.171863,-115.146721,"[{'label': 'display', 'lat': 36.17186295186981...",537,89101,US,Las Vegas,NV,United States,"[1 S Main St, Las Vegas, NV 89101, United States]",4edd4eb1754a23a1927a192e
4,Hollywood Palace Hotel (PieceOfMe),Hotel,NaN,NaN,36.168302,-115.145427,"[{'label': 'display', 'lat': 36.168302, 'lng':...",301,NaN,US,Las Vegas,NV,United States,"[Las Vegas, NV, United States]",565f0c61498e5954cc590d8a


In [72]:
clean_dataframe2= clean_dataframe.drop(['cc', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_dataframe2

,name,categories,address,lat,lng,postalCode,city,state
0,Golden Nugget Hotel & Casino,Casino,129 Fremont St,36.170222,-115.144969,89101,Las Vegas,NV
1,William Hill Sports Book - The Plaza Hotel and...,Casino,1 S Main St,36.171239,-115.146849,89101,Las Vegas,NV
2,Golden Gate Hotel & Casino,Casino,1 Fremont Street,36.171263,-115.146209,89101,Las Vegas,NV
3,LasVegasShows.com Ticket Kiosk - Plaza Hotel a...,Miscellaneous Shop,1 S Main St,36.171863,-115.146721,89101,Las Vegas,NV
4,Hollywood Palace Hotel (PieceOfMe),Hotel,NaN,36.168302,-115.145427,NaN,Las Vegas,NV
5,Hertz-Plaza Hotel & Casino,Rental Car Location,NaN,36.171913,-115.146420,NaN,Las Vegas,NV
6,Plaza Hotel Wedding Chapel and Florist,Church,1 South Main Street,36.171607,-115.146503,89101,Las Vegas,NV
7,LAS VEGAS Royal Hotel,Nightclub,NaN,36.168148,-115.141708,NaN,Las Vegas,NV
8,Zaba's Mexican Grill @ Plaza Hotel,Mexican Restaurant,NaN,36.171871,-115.146238,89101,Las Vegas,NV
9,California Hotel Gym,Gym / Fitness Center,NaN,36.172263,-115.144729,89101,Las Vegas,NV


### Now that we have a dataframe with only RELEVANT information, let's now remove missing information so that out dataframe will be immaculate.

In [73]:
# delete rows with none values
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_dataframe3

,name,categories,address,lat,lng,postalCode,city,state
0,Golden Nugget Hotel & Casino,Casino,129 Fremont St,36.170222,-115.144969,89101,Las Vegas,NV
1,William Hill Sports Book - The Plaza Hotel and...,Casino,1 S Main St,36.171239,-115.146849,89101,Las Vegas,NV
2,Golden Gate Hotel & Casino,Casino,1 Fremont Street,36.171263,-115.146209,89101,Las Vegas,NV
3,LasVegasShows.com Ticket Kiosk - Plaza Hotel a...,Miscellaneous Shop,1 S Main St,36.171863,-115.146721,89101,Las Vegas,NV
6,Plaza Hotel Wedding Chapel and Florist,Church,1 South Main Street,36.171607,-115.146503,89101,Las Vegas,NV
10,Golden Nugget Pool,Hotel Pool,129 Fremont St,36.170005,-115.145647,89101,Las Vegas,NV
11,Plaza Casino,Casino,1 S Main St,36.171398,-115.146687,89101,Las Vegas,NV
12,Oscar's Steakhouse,Lounge,1 S Main St,36.171583,-115.146606,89101,Las Vegas,NV
13,Rush Tower,Hotel,129 Fremont St,36.170300,-115.145957,89101,Las Vegas,NV
14,Golden Nugget Buffet,Buffet,129 Fremont St,36.170028,-115.145596,89101,Las Vegas,NV


### It is important to understand that our goal here is only to find hotels for the tourist, so let's not be lenient and let's remove all "hotels" that aren't specifically categorized as a hotel.

In [88]:
# delete rows which its category is not Hotel or Event Space
array= ['Hotel']
hotel_dataframe= clean_dataframe3.loc[clean_dataframe3['categories'].isin(array)]
hotel_dataframe

,name,categories,address,lat,lng,postalCode,city,state
13,Rush Tower,Hotel,129 Fremont St,36.1703,-115.145957,89101,Las Vegas,NV


### Let's now repeat the exact same process used for hotels on casinos as Las Vegas's main attraction is all the casinos

In [75]:
search_query = 'Casino'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=2XM3I5ZAP13OTFTXRDRE03CUCDA5LWAPRFVJXX5BPO0CIOWX&client_secret=GK0UWICKTRQ4QPXTWV5DFBLVZQ3WJGZHDLX31OMRRNK0ZYT2&ll=36.1672559,-115.1485163&v=20200425&query=Casino&radius=10000&limit=30'

In [76]:
# Send the GET Request and examine the results
presults = requests.get(url).json()

In [82]:
# assign relevant part of JSON to venues
venues = presults['response']['venues']

# tranform venues into a dataframe
casino_dataframe = json_normalize(venues)
casino_dataframe.head()

C:\Users\vkogi\Anaconda3\envs\python-cvcourse\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,venuePage.id
0,4fb27eb5e4b0e1e9692e0461,Casino Center Boulevard & Lewis Avenue,"[{'id': '52f2ab2ebcbc57f1066b8b4c', 'name': 'I...",v-1587857317,False,S Casino Center Blvd,at Lewis Ave,36.167205,-115.146343,"[{'label': 'display', 'lat': 36.1672054383697,...",195,89101,US,Las Vegas,NV,United States,"[S Casino Center Blvd (at Lewis Ave), Las Vega...",NaN,NaN
1,507b8ea2e4b047ff2d00fb35,Casino Center Boulevard & Bonneville Avenue,"[{'id': '52f2ab2ebcbc57f1066b8b4c', 'name': 'I...",v-1587857317,False,S Casino Center Blvd,at Bonneville Ave,36.164899,-115.147882,"[{'label': 'display', 'lat': 36.16489931459548...",268,89101,US,Las Vegas,NV,United States,"[S Casino Center Blvd (at Bonneville Ave), Las...",NaN,NaN
2,49de9ab7f964a5208a601fe3,Four Queens Hotel & Casino,"[{'id': '4bf58dd8d48988d17c941735', 'name': 'C...",v-1587857317,False,202 Fremont St,at Casino Center Blvd,36.170266,-115.143992,"[{'label': 'display', 'lat': 36.170266, 'lng':...",526,89101,US,Las Vegas,NV,United States,"[202 Fremont St (at Casino Center Blvd), Las V...",Downtown Las Vegas,NaN
3,45a90a24f964a52020411fe3,Golden Nugget Hotel & Casino,"[{'id': '4bf58dd8d48988d17c941735', 'name': 'C...",v-1587857317,False,129 Fremont St,at S Casino Center Blvd,36.170222,-115.144969,"[{'label': 'display', 'lat': 36.170222, 'lng':...",458,89101,US,Las Vegas,NV,United States,"[129 Fremont St (at S Casino Center Blvd), Las...",NaN,NaN
4,49dea2eaf964a52093601fe3,Plaza Casino,"[{'id': '4bf58dd8d48988d17c941735', 'name': 'C...",v-1587857317,False,1 S Main St,NaN,36.171398,-115.146687,"[{'label': 'display', 'lat': 36.17139842699489...",489,89101,US,Las Vegas,NV,United States,"[1 S Main St, Las Vegas, NV 89101, United States]",NaN,407572130


In [83]:
# keep only columns that include venue name, and anything that is associated with location
casino_clean_columns = ['name', 'categories'] + [col for col in casino_dataframe.columns if col.startswith('location.')]+ ['id']
clean_casino_dataframe = casino_dataframe.loc[:,casino_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

# filter the category for each row
clean_casino_dataframe['categories'] = clean_casino_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_casino_dataframe.columns = [column.split('.')[-1] for column in clean_casino_dataframe.columns]

clean_casino_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Casino Center Boulevard & Lewis Avenue,Intersection,S Casino Center Blvd,at Lewis Ave,36.167205,-115.146343,"[{'label': 'display', 'lat': 36.1672054383697,...",195,89101,US,Las Vegas,NV,United States,"[S Casino Center Blvd (at Lewis Ave), Las Vega...",NaN,4fb27eb5e4b0e1e9692e0461
1,Casino Center Boulevard & Bonneville Avenue,Intersection,S Casino Center Blvd,at Bonneville Ave,36.164899,-115.147882,"[{'label': 'display', 'lat': 36.16489931459548...",268,89101,US,Las Vegas,NV,United States,"[S Casino Center Blvd (at Bonneville Ave), Las...",NaN,507b8ea2e4b047ff2d00fb35
2,Four Queens Hotel & Casino,Casino,202 Fremont St,at Casino Center Blvd,36.170266,-115.143992,"[{'label': 'display', 'lat': 36.170266, 'lng':...",526,89101,US,Las Vegas,NV,United States,"[202 Fremont St (at Casino Center Blvd), Las V...",Downtown Las Vegas,49de9ab7f964a5208a601fe3
3,Golden Nugget Hotel & Casino,Casino,129 Fremont St,at S Casino Center Blvd,36.170222,-115.144969,"[{'label': 'display', 'lat': 36.170222, 'lng':...",458,89101,US,Las Vegas,NV,United States,"[129 Fremont St (at S Casino Center Blvd), Las...",NaN,45a90a24f964a52020411fe3
4,Plaza Casino,Casino,1 S Main St,NaN,36.171398,-115.146687,"[{'label': 'display', 'lat': 36.17139842699489...",489,89101,US,Las Vegas,NV,United States,"[1 S Main St, Las Vegas, NV 89101, United States]",NaN,49dea2eaf964a52093601fe3


In [84]:
# delete unnecessary columns
clean_casino_dataframe2= clean_park_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood','id'], axis=1)
clean_casino_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Casino Center Boulevard & Lewis Avenue,Intersection,S Casino Center Blvd,36.167205,-115.146343,89101,NV
1,Casino Center Boulevard & Bonneville Avenue,Intersection,S Casino Center Blvd,36.164899,-115.147882,89101,NV
2,Four Queens Hotel & Casino,Casino,202 Fremont St,36.170266,-115.143992,89101,NV
3,Golden Nugget Hotel & Casino,Casino,129 Fremont St,36.170222,-115.144969,89101,NV
4,Plaza Casino,Casino,1 S Main St,36.171398,-115.146687,89101,NV
5,The D Las Vegas Casino Hotel,Casino,301 Fremont St,36.169655,-115.142732,89101,NV
6,California Hotel & Casino,Hotel,12 E Ogden Ave,36.173029,-115.144199,89101,NV
7,Fremont Hotel & Casino,Casino,200 Fremont St,36.171017,-115.143444,89101,NV
8,William Hill Sports Book - The Plaza Hotel and...,Casino,1 S Main St,36.171239,-115.146849,89101,NV
9,"Main Street Station Casino, Brewery & Hotel",Casino,200 N Main St,36.174195,-115.145155,89101,NV


In [85]:
# delete rows with none values
clean_casino_dataframe3 = clean_casino_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_casino_dataframe3

,name,categories,address,lat,lng,postalCode,state
0,Casino Center Boulevard & Lewis Avenue,Intersection,S Casino Center Blvd,36.167205,-115.146343,89101,NV
1,Casino Center Boulevard & Bonneville Avenue,Intersection,S Casino Center Blvd,36.164899,-115.147882,89101,NV
2,Four Queens Hotel & Casino,Casino,202 Fremont St,36.170266,-115.143992,89101,NV
3,Golden Nugget Hotel & Casino,Casino,129 Fremont St,36.170222,-115.144969,89101,NV
4,Plaza Casino,Casino,1 S Main St,36.171398,-115.146687,89101,NV
5,The D Las Vegas Casino Hotel,Casino,301 Fremont St,36.169655,-115.142732,89101,NV
6,California Hotel & Casino,Hotel,12 E Ogden Ave,36.173029,-115.144199,89101,NV
7,Fremont Hotel & Casino,Casino,200 Fremont St,36.171017,-115.143444,89101,NV
8,William Hill Sports Book - The Plaza Hotel and...,Casino,1 S Main St,36.171239,-115.146849,89101,NV
9,"Main Street Station Casino, Brewery & Hotel",Casino,200 N Main St,36.174195,-115.145155,89101,NV


In [89]:
# delete rows which its category is not Casino
df_casino = clean_casino_dataframe3[clean_casino_dataframe3.categories == 'Casino']
df_casino

,name,categories,address,lat,lng,postalCode,state
2,Four Queens Hotel & Casino,Casino,202 Fremont St,36.170266,-115.143992,89101,NV
3,Golden Nugget Hotel & Casino,Casino,129 Fremont St,36.170222,-115.144969,89101,NV
4,Plaza Casino,Casino,1 S Main St,36.171398,-115.146687,89101,NV
5,The D Las Vegas Casino Hotel,Casino,301 Fremont St,36.169655,-115.142732,89101,NV
7,Fremont Hotel & Casino,Casino,200 Fremont St,36.171017,-115.143444,89101,NV
8,William Hill Sports Book - The Plaza Hotel and...,Casino,1 S Main St,36.171239,-115.146849,89101,NV
9,"Main Street Station Casino, Brewery & Hotel",Casino,200 N Main St,36.174195,-115.145155,89101,NV
11,Sports Book at Fremont Casino,Casino,200 Fremont St,36.170845,-115.143974,89101,NV
14,"The STRAT Hotel, Casino and Skypod",Casino,2000 Las Vegas Blvd S,36.147964,-115.156084,89104,NV
19,El Cortez Hotel & Casino,Casino,600 Fremont St,36.168892,-115.139025,89101,NV


### Let's now repeat the exact same process used for hotels on restaurants as it is very important that the tourist has places to eat.

In [91]:
search_query = 'Restaurant'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=2XM3I5ZAP13OTFTXRDRE03CUCDA5LWAPRFVJXX5BPO0CIOWX&client_secret=GK0UWICKTRQ4QPXTWV5DFBLVZQ3WJGZHDLX31OMRRNK0ZYT2&ll=36.1672559,-115.1485163&v=20200425&query=Restaurant&radius=10000&limit=30'

In [93]:
# Send the GET Request and examine the results
presults = requests.get(url).json()

In [95]:
# assign relevant part of JSON to venues
venues = presults['response']['venues']

# tranform venues into a dataframe
Restaurant_dataframe = json_normalize(venues)
Restaurant_dataframe.head()

C:\Users\vkogi\Anaconda3\envs\python-cvcourse\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.country,location.formattedAddress,venuePage.id,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,location.neighborhood
0,4b88548cf964a520b2ef31e3,Chart House Restaurant,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",v-1587857621,False,129 Fremont St,at Golden Nugget Hotel,36.170159,-115.145008,"[{'label': 'display', 'lat': 36.17015864255055...",...,United States,"[129 Fremont St (at Golden Nugget Hotel), Las ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4b102f49f964a520c66a23e3,Doña María Tamales Restaurant,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",v-1587857621,False,910 Las Vegas Blvd S,Charleston Blvd.,36.159566,-115.147341,"[{'label': 'display', 'lat': 36.15956607123959...",...,United States,"[910 Las Vegas Blvd S (Charleston Blvd.), Las ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,41326e00f964a52035151fe3,Triple 7 Restaurant & Brewery,"[{'id': '4bf58dd8d48988d155941735', 'name': 'G...",v-1587857621,False,200 N Main St,at Stewart Ave,36.174490,-115.144820,"[{'label': 'display', 'lat': 36.17449001007171...",...,United States,"[200 N Main St (at Stewart Ave), Las Vegas, NV...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4b8a987df964a5202c7432e3,E-jo Korean Restaurant,"[{'id': '4bf58dd8d48988d113941735', 'name': 'K...",v-1587857621,False,700 E Sahara Ave,NaN,36.144205,-115.145276,"[{'label': 'display', 'lat': 36.14420505435932...",...,United States,"[700 E Sahara Ave, Las Vegas, NV 89104, United...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,521be6ca11d22a8b5a7d6199,Lev Restaurant group (Jamba Juice),"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1587857621,False,NaN,NaN,36.164756,-115.151244,"[{'label': 'display', 'lat': 36.164756, 'lng':...",...,United States,"[Las Vegas, NV, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
# keep only columns that include venue name, and anything that is associated with location
Restaurant_clean_columns = ['name', 'categories'] + [col for col in Restaurant_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Restaurant_dataframe = Restaurant_dataframe.loc[:,Restaurant_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

# filter the category for each row
clean_Restaurant_dataframe['categories'] = clean_Restaurant_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Restaurant_dataframe.columns = [column.split('.')[-1] for column in clean_Restaurant_dataframe.columns]

clean_Restaurant_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Chart House Restaurant,Seafood Restaurant,129 Fremont St,at Golden Nugget Hotel,36.170159,-115.145008,"[{'label': 'display', 'lat': 36.17015864255055...",451,89101,US,Las Vegas,NV,United States,"[129 Fremont St (at Golden Nugget Hotel), Las ...",NaN,4b88548cf964a520b2ef31e3
1,Doña María Tamales Restaurant,Mexican Restaurant,910 Las Vegas Blvd S,Charleston Blvd.,36.159566,-115.147341,"[{'label': 'display', 'lat': 36.15956607123959...",862,89101,US,Las Vegas,NV,United States,"[910 Las Vegas Blvd S (Charleston Blvd.), Las ...",NaN,4b102f49f964a520c66a23e3
2,Triple 7 Restaurant & Brewery,Gastropub,200 N Main St,at Stewart Ave,36.174490,-115.144820,"[{'label': 'display', 'lat': 36.17449001007171...",871,89101,US,Las Vegas,NV,United States,"[200 N Main St (at Stewart Ave), Las Vegas, NV...",NaN,41326e00f964a52035151fe3
3,E-jo Korean Restaurant,Korean Restaurant,700 E Sahara Ave,NaN,36.144205,-115.145276,"[{'label': 'display', 'lat': 36.14420505435932...",2582,89104,US,Las Vegas,NV,United States,"[700 E Sahara Ave, Las Vegas, NV 89104, United...",NaN,4b8a987df964a5202c7432e3
4,Lev Restaurant group (Jamba Juice),Office,NaN,NaN,36.164756,-115.151244,"[{'label': 'display', 'lat': 36.164756, 'lng':...",370,NaN,US,Las Vegas,NV,United States,"[Las Vegas, NV, United States]",NaN,521be6ca11d22a8b5a7d6199


In [97]:
# delete unnecessary columns
clean_Restaurant_dataframe2= clean_Restaurant_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_Restaurant_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Chart House Restaurant,Seafood Restaurant,129 Fremont St,36.170159,-115.145008,89101,NV
1,Doña María Tamales Restaurant,Mexican Restaurant,910 Las Vegas Blvd S,36.159566,-115.147341,89101,NV
2,Triple 7 Restaurant & Brewery,Gastropub,200 N Main St,36.174490,-115.144820,89101,NV
3,E-jo Korean Restaurant,Korean Restaurant,700 E Sahara Ave,36.144205,-115.145276,89104,NV
4,Lev Restaurant group (Jamba Juice),Office,NaN,36.164756,-115.151244,NaN,NV
5,Grotto Italian Restaurant,Italian Restaurant,129 Fremont St,36.169880,-115.145702,89101,NV
6,Restaurant Equipment Supply Co,Miscellaneous Shop,1300 E Charleston Blvd,36.158167,-115.135439,89104,NV
7,Mariscos 7 Mares Mexican Restaurant,Mexican Restaurant,2000 E. Charleston Blvd.,36.158544,-115.121042,89104,NV
8,The Fitz- Restaurant,Steakhouse,NaN,36.169941,-115.142106,NaN,Nevada
9,Jin Mee Korean BBQ Restaurant,Korean Restaurant,953 E Sahara Ave,36.142573,-115.142182,89104,NV


In [105]:
# delete rows with none values
df_Restaurant = clean_Restaurant_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant

,name,categories,address,lat,lng,postalCode,state
0,Chart House Restaurant,Seafood Restaurant,129 Fremont St,36.170159,-115.145008,89101,NV
1,Doña María Tamales Restaurant,Mexican Restaurant,910 Las Vegas Blvd S,36.159566,-115.147341,89101,NV
2,Triple 7 Restaurant & Brewery,Gastropub,200 N Main St,36.174490,-115.144820,89101,NV
3,E-jo Korean Restaurant,Korean Restaurant,700 E Sahara Ave,36.144205,-115.145276,89104,NV
5,Grotto Italian Restaurant,Italian Restaurant,129 Fremont St,36.169880,-115.145702,89101,NV
6,Restaurant Equipment Supply Co,Miscellaneous Shop,1300 E Charleston Blvd,36.158167,-115.135439,89104,NV
7,Mariscos 7 Mares Mexican Restaurant,Mexican Restaurant,2000 E. Charleston Blvd.,36.158544,-115.121042,89104,NV
9,Jin Mee Korean BBQ Restaurant,Korean Restaurant,953 E Sahara Ave,36.142573,-115.142182,89104,NV
10,Restaurant Guy Savoy,French Restaurant,3570 Las Vegas Blvd S,36.115085,-115.175120,89109,NV
11,Makino Premium Restaurant,Food,775 S Grand Central Pkwy,36.162569,-115.156140,89106,NV


### Let's now repeat the exact same process used for hotels on shopping as it is very important that the tourist has places to shop and buy merchandise / souvenirs.

In [118]:
# search for Shopping
search_query = 'Mall'
radius = 1000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=2XM3I5ZAP13OTFTXRDRE03CUCDA5LWAPRFVJXX5BPO0CIOWX&client_secret=GK0UWICKTRQ4QPXTWV5DFBLVZQ3WJGZHDLX31OMRRNK0ZYT2&ll=36.1672559,-115.1485163&v=20200425&query=Mall&radius=1000&limit=30'

In [119]:
# Send the GET Request and examine the results
sresults = requests.get(url).json()

In [121]:
# assign relevant part of JSON to venues
venues = sresults['response']['venues']

# tranform venues into a dataframe
Shopping_dataframe = json_normalize(venues)
Shopping_dataframe.head()

C:\Users\vkogi\Anaconda3\envs\python-cvcourse\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,440ce8b1f964a52085301fe3,Neonopolis Mall,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1587858120,False,450 Fremont St,36.169998,-115.140915,"[{'label': 'display', 'lat': 36.1699983753398,...",748,89101,US,Las Vegas,NV,United States,"[450 Fremont St, Las Vegas, NV 89101, United S...",NaN
1,53d03ee4498e01c344b0af38,The Mob Museum Retail&Ticketing Kiosk At The P...,"[{'id': '4bf58dd8d48988d190941735', 'name': 'H...",v-1587858120,False,NaN,36.165560,-115.158528,"[{'label': 'display', 'lat': 36.16556010249218...",919,NaN,US,Las Vegas,NV,United States,"[Las Vegas, NV, United States]",NaN
2,4a4524a5f964a520dea71fe3,Las Vegas North Premium Outlets,"[{'id': '5744ccdfe4b0c0459246b4df', 'name': 'O...",v-1587858120,False,875 S Grand Central Pkwy,36.164049,-115.157513,"[{'label': 'display', 'lat': 36.1640488, 'lng'...",883,89106,US,Las Vegas,NV,United States,[875 S Grand Central Pkwy (at W Bonneville Ave...,at W Bonneville Ave
3,50a5ab07e4b0540fdfc54260,Mallorie Nasrallah Photography,"[{'id': '4bf58dd8d48988d1e2931735', 'name': 'A...",v-1587858120,False,1115 S Casino Center Blvd #8,36.158797,-115.152066,"[{'label': 'display', 'lat': 36.15879659630171...",994,89104,US,Las Vegas,NV,United States,"[1115 S Casino Center Blvd #8 (Charleston), La...",Charleston
4,4c3a6a6a3849c92856fec2b1,Zumiez,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",v-1587858120,False,875 S Grand Central Pkwy #1132,36.164119,-115.157015,"[{'label': 'display', 'lat': 36.16411904053578...",839,89106,US,Las Vegas,NV,United States,"[875 S Grand Central Pkwy #1132, Las Vegas, NV...",NaN


In [123]:
# keep only columns that include venue name, and anything that is associated with location
Shopping_clean_columns = ['name', 'categories'] + [col for col in Shopping_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Shopping_dataframe = Shopping_dataframe.loc[:,Shopping_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list5 = row['categories']
    except:
        categories_list5 = row['venue.categories']
        
    if len(categories_list5) == 0:
        return None
    else:
        return categories_list5[0]['name']

# filter the category for each row
clean_Shopping_dataframe['categories'] = clean_Shopping_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Shopping_dataframe.columns = [column.split('.')[-1] for column in clean_Shopping_dataframe.columns]

clean_Shopping_dataframe.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Neonopolis Mall,Shopping Mall,450 Fremont St,36.169998,-115.140915,"[{'label': 'display', 'lat': 36.1699983753398,...",748,89101,US,Las Vegas,NV,United States,"[450 Fremont St, Las Vegas, NV 89101, United S...",NaN,440ce8b1f964a52085301fe3
1,The Mob Museum Retail&Ticketing Kiosk At The P...,History Museum,NaN,36.165560,-115.158528,"[{'label': 'display', 'lat': 36.16556010249218...",919,NaN,US,Las Vegas,NV,United States,"[Las Vegas, NV, United States]",NaN,53d03ee4498e01c344b0af38
2,Las Vegas North Premium Outlets,Outlet Mall,875 S Grand Central Pkwy,36.164049,-115.157513,"[{'label': 'display', 'lat': 36.1640488, 'lng'...",883,89106,US,Las Vegas,NV,United States,[875 S Grand Central Pkwy (at W Bonneville Ave...,at W Bonneville Ave,4a4524a5f964a520dea71fe3
3,Mallorie Nasrallah Photography,Art Gallery,1115 S Casino Center Blvd #8,36.158797,-115.152066,"[{'label': 'display', 'lat': 36.15879659630171...",994,89104,US,Las Vegas,NV,United States,"[1115 S Casino Center Blvd #8 (Charleston), La...",Charleston,50a5ab07e4b0540fdfc54260
4,Zumiez,Clothing Store,875 S Grand Central Pkwy #1132,36.164119,-115.157015,"[{'label': 'display', 'lat': 36.16411904053578...",839,89106,US,Las Vegas,NV,United States,"[875 S Grand Central Pkwy #1132, Las Vegas, NV...",NaN,4c3a6a6a3849c92856fec2b1


In [127]:
# delete unnecessary columns
df_shopping= clean_Shopping_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
df_shopping

,name,categories,address,lat,lng,postalCode,state
0,Neonopolis Mall,Shopping Mall,450 Fremont St,36.169998,-115.140915,89101,NV
1,The Mob Museum Retail&Ticketing Kiosk At The P...,History Museum,NaN,36.165560,-115.158528,NaN,NV
2,Las Vegas North Premium Outlets,Outlet Mall,875 S Grand Central Pkwy,36.164049,-115.157513,89106,NV
3,Mallorie Nasrallah Photography,Art Gallery,1115 S Casino Center Blvd #8,36.158797,-115.152066,89104,NV
4,Zumiez,Clothing Store,875 S Grand Central Pkwy #1132,36.164119,-115.157015,89106,NV
5,PacSun,Clothing Store,505 S Grand Central Pkwy #1405,36.164968,-115.157395,89106,NV
6,Starbucks,Coffee Shop,855 S Grand Central Pkwy,36.164715,-115.156604,89106,NV
7,The PUMA Outlet,Shoe Store,755 S Grand Central Pkwy Ste 1415,36.165267,-115.157236,89106,NV
8,Nine West,Shoe Store,3200 Las Vegas Blvd S #2360,36.162601,-115.158138,89109,NV
9,Auntie Anne's,Snack Place,775 S Grand Central Pkwy #1320,36.164787,-115.157521,89106,NV


## Methodology <a name="methodology"></a>

The methodology used after sorting and creating the data is the use a folium map and map out all areas of hotels, casinos, restaurants, and malls to understand how exactly hotels are clustered around tourist sites. When plotting the coordinates on folium, I will make the color of the hotel coordinates red and this way, it will be easy to see how exactly Intrepid Travel can map out tourist packages according to how tourist sites are clustered.

I will first create a neighborhood dataframe of all locations
<br></br>
I will then add an if statement to isolate hotels
<br></br>
Lastly, we can analyze with the eye, the clusters in Las Vegas around Hotels.

## Analysis <a name="analysis"></a>

In [138]:
# create dataframe of hotels, casinos, malls, and restaurants
hotel_neighbourhood_df = pd.concat([hotel_dataframe, df_casino, df_shopping, df_Restaurant], ignore_index=True)
hotel_neighbourhood_df

,name,categories,address,lat,lng,postalCode,city,state
0,Rush Tower,Hotel,129 Fremont St,36.170300,-115.145957,89101,Las Vegas,NV
1,Four Queens Hotel & Casino,Casino,202 Fremont St,36.170266,-115.143992,89101,NaN,NV
2,Golden Nugget Hotel & Casino,Casino,129 Fremont St,36.170222,-115.144969,89101,NaN,NV
3,Plaza Casino,Casino,1 S Main St,36.171398,-115.146687,89101,NaN,NV
4,The D Las Vegas Casino Hotel,Casino,301 Fremont St,36.169655,-115.142732,89101,NaN,NV
5,Fremont Hotel & Casino,Casino,200 Fremont St,36.171017,-115.143444,89101,NaN,NV
6,William Hill Sports Book - The Plaza Hotel and...,Casino,1 S Main St,36.171239,-115.146849,89101,NaN,NV
7,"Main Street Station Casino, Brewery & Hotel",Casino,200 N Main St,36.174195,-115.145155,89101,NaN,NV
8,Sports Book at Fremont Casino,Casino,200 Fremont St,36.170845,-115.143974,89101,NaN,NV
9,"The STRAT Hotel, Casino and Skypod",Casino,2000 Las Vegas Blvd S,36.147964,-115.156084,89104,NaN,NV


In [137]:
# Generate map to visualize hotel neighbourhood including shopping stores and Cafeteria 
hotel_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(hotel_neighbourhood_df['lat'], hotel_neighbourhood_df['lng'], 
                                           hotel_neighbourhood_df['name'], hotel_neighbourhood_df['categories'],\
                                               hotel_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    if categories == 'Casino':
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7,
            parse_html=False).add_to(hotel_map) 
    else:
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.7,
            parse_html=False).add_to(hotel_map)
    
hotel_map

## Results and Discussion <a name="results"></a>

It can be seen by zooming in and analyzing the clusters that Fremont Secret Experience Area has a high cluster of hotels with a lower cluster of amenities making it possibility crowded, overbooks, and unappealing for tourist travel. It can also be seen that the Las Vegas Premium Outlets are a very famous chain of outlet mall with shops and restaurants, however, with not many hotels nearby. An area that seems to balance the hotel with amenities is the Las Vegas Convention Center Area where it would be preferred for tourist travels.

## Conclusion <a name="conclusion"></a>

The purpose of this project was to identify areas of proper hotel to amenity balance for Intrepid Travels such that tourists will be happy and have a good vacation in Las Vegas. It was identified that the Fremont Secret Experience Area is a very famous area due to all the major hotel establishments however it may be too crowded. It was also determined that the lowest hotel to amenity ratio cluster was at the Las Vegas Premium Outlets, however, was zero making it also dislikable. A good in between found was areas near the Las Vegas COnvention Center.